In [90]:
!pip install sktime

  Obtaining dependency information for sktime from https://files.pythonhosted.org/packages/ce/2a/039b8045fcd38a69bee49f9997a81a74296b9dc52d44ea15460a82c80d92/sktime-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-base<0.7.0 from https://files.pythonhosted.org/packages/ba/5d/cf3d9ae370bc67f6d81f54a7f80656ef5b88635f67fb2339f1a0eb666551/scikit_base-0.6.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 37.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 1.9 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [28]:
!pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/7c/55/b3432f43d6d7fee999bb23a547820d74c48ec540f5f7842e41aa5d8d5f3a/datasets-2.14.6-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/80/8a/1dd41557883b6196f8f092011a5c1f72d4d44cf36d7b67d4a5efe3127949/xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for aiohttp from https://files.pythonhosted.org/packages/41/8e/4c48881316bbced3d13089c4d0df4be321ce79a0c695d82dee9996aaf56b/

In [2]:
!pip install 'transformers[torch]'

  Obtaining dependency information for torch!=1.12.0,>=1.10 from https://files.pythonhosted.org/packages/6d/13/b5e8bacd980b2195f8a1741ce11cbb9146568607795d5e4ff510dcff1064/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Obtaining dependency information for nvidia-cudnn-cu12==8.9.2.26 from https://files.pythonhosted.org/packages/ff/74/a2e2be7fb83aaedec84f391f082cf765dfb635e7caa9b49065f73e4835d8/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using 

In [3]:
!pip install huggingface_hub


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [128]:
from huggingface_hub import hf_hub_download
import torch
from transformers import AutoformerForPrediction,AutoformerModel
from datasets import load_dataset
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError, MeanAbsoluteScaledError, MedianAbsoluteScaledError 
import pandas as pd

In [46]:
file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

In [107]:
file

'/root/.cache/huggingface/hub/datasets--hf-internal-testing--tourism-monthly-batch/snapshots/81c7ee3cf3317e51beb97327df55926cd5bbfadb/train-batch.pt'

In [77]:
batch["future_values"].size()

torch.Size([64, 24])

In [47]:
model = AutoformerForPrediction.from_pretrained("huggingface/autoformer-tourism-monthly")

In [57]:
# during training, one provides both past and future values
# as well as possible additional features
outputs = model.generate(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    future_time_features=batch["future_time_features"],
)

In [78]:
outputs.sequences.size()

torch.Size([64, 100, 24])

In [62]:
outputs

SampleTSPredictionOutput(sequences=tensor([[[ 2.6442e+04,  2.8032e+04,  3.2205e+04,  ...,  3.1838e+04,
           3.7873e+04,  4.3878e+04],
         [ 2.1588e+04,  2.5244e+04,  3.2670e+04,  ...,  3.5131e+04,
           4.1842e+04,  3.7473e+04],
         [ 3.0159e+04,  2.6721e+04,  2.4896e+04,  ...,  2.7859e+04,
           3.2314e+04,  3.4371e+04],
         ...,
         [ 2.4882e+04,  2.5699e+04,  3.4027e+04,  ...,  3.0273e+04,
           5.6476e+04,  2.7183e+04],
         [ 2.3365e+04,  3.0658e+04,  3.4204e+04,  ...,  2.9690e+04,
           3.3254e+04,  3.0976e+04],
         [ 3.1808e+04,  3.2179e+04,  4.1334e+04,  ...,  3.6912e+04,
           4.1808e+04,  3.3516e+04]],

        [[ 2.3285e+04,  2.8122e+04,  1.2016e+04,  ...,  1.2994e+04,
           1.4527e+04,  2.3341e+04],
         [ 1.9249e+04,  2.7809e+04,  3.1431e+04,  ...,  1.7513e+04,
           1.7256e+04,  2.2740e+04],
         [ 1.2957e+04,  3.2961e+04,  4.9138e+04,  ...,  2.0615e+04,
           3.0243e+04, -1.2792e+04],
    

In [94]:
values = batch["future_values"].tolist()

In [96]:
preds = outputs.sequences.mean(dim=1).tolist()

In [98]:
mape = MeanAbsolutePercentageError()
mase = MeanAbsoluteScaledError()
mdase = MedianAbsoluteScaledError()

In [152]:
series_mape = []
series_mase = []
for i in range(len(values)):
    series_mape.append(mape(y_true=pd.Series(values[i]),y_pred=pd.Series(preds[i])))
    series_mase.append(mase(y_true=pd.Series(values[i]),y_pred=pd.Series(preds[i]),y_train=pd.Series(batch["past_values"].tolist()[i])))

In [105]:
import numpy as np

In [106]:
np.mean(series_mape)

0.21367555818273784

In [153]:
np.mean(series_mase)

1.2882155777463482